## Extract Poses from Amass Dataset

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import rotation_conversions as rc


from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [ ]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [1]:
import smplx
smplx_bm = smplx.create(
            "../datasets/hub/smplx_models/", 
            model_type='smplx',
            gender='NEUTRAL_2020', 
            use_face_contour=False,
            num_betas=300,
            num_expression_coeffs=100, 
            ext='npz',
            use_pca=False,
        ).cuda().eval()

def load_amass(data):
    ## 这个是用来
    # 修改amass数据里面的朝向，原本在blender里面是Z轴向上，目标是Y轴向上，当时面向目前没改
    
    data_dict = {key: data[key] for key in data}
    frames = data_dict['poses'].shape[0]
    # b = data_dict['poses'][...,:3]
    # b = rc.axis_angle_to_matrix(torch.from_numpy(b))
    # rot_matrix = np.array([[1.0, 0.0, 0.0], [0.0 , 0.0, 1.0], [0.0, -1.0, 0.0]])
    # c = np.einsum('ij,kjl->kil',rot_matrix,b)
    # c = rc.matrix_to_axis_angle(torch.from_numpy(c))
    # data_dict['poses'][...,:3] = c
    
    # trans_matrix1 = np.array([[1.0, 0.0, 0.0], [0.0 , 0.0, -1.0], [0.0, 1.0, 0.0]])
    # data_dict['trans'] = np.einsum("bi,ij->bj",data_dict['trans'],trans_matrix1)
    
    betas300 = np.zeros(300)
    betas300[:16] = data_dict['betas']
    data_dict['betas'] = betas300
    data_dict["expressions"] = np.zeros((frames,100))
    return data_dict

def load_beat(data):
    
    data_dict = {key: data[key] for key in data}
    frames = data_dict['poses'].shape[0]

    b = data_dict['poses'][...,:3]
    b = rc.axis_angle_to_matrix(torch.from_numpy(b))
    rot_matrix = np.array([[1.0, 0.0, 0.0], [0.0 , 0.0, -1.0], [0.0, 1.0, 0.0]])
    

    c = np.einsum('ij,kjl->kil',rot_matrix,b)
    
    
    c = rc.matrix_to_axis_angle(torch.from_numpy(c))
    data_dict['poses'][...,:3] = c
    
    trans_matrix1 = np.array([[1.0, 0.0, 0.0], [0.0 , 0.0, 1.0], [0.0, -1.0, 0.0]])
    
    #trans_matrix2 = np.array([[-1.0, 0.0, 0.0], [0.0 , -1.0, 0.0], [0.0, 0.0, 1.0]])
    #trans_matrix2 = np.array([[1.0, 0.0, 0.0], [0.0 , -1.0, 0.0], [0.0, 0.0, -1.0]])
    #data_dict['trans'] = np.einsum("bi,ij->bj",data_dict['trans'],trans_matrix)
    data_dict['trans'] = np.einsum("bi,ij->bj",data_dict['trans'],trans_matrix1)

    return data_dict


In [ ]:
paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk('./beat_data'):
#     print(root, dirs, files)
#     for folder in dirs:
#         folders.append(os.path.join(root, folder))
    folders.append(root)
    for name in files:
        paths.append(os.path.join(root, name))

In [ ]:
save_root = './pose_data'
save_folders = [folder.replace('./beat_data', './pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)

In [ ]:
trans_matrix = np.array([[1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0],
                            [0.0, 1.0, 0.0]])
ex_fps = 30
target_joint = list(range(22))+list(range(25,55))
def amass_to_pose(src_path, save_path):
    bdata = np.load(src_path, allow_pickle=True)
    if len(bdata.files)==6:
        #print(f"# ---- state file ---- #")
        return 0
    bdata = load_beat(bdata)
    fps = 0
    try:
        fps = bdata['mocap_frame_rate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps
    fId = 0 # frame id of the mocap sequence
    pose_seq = []    
    
    betas = bdata['betas'].reshape(1,300)
    betas = np.tile(betas, (frame_number, 1))
    poses = bdata['poses']
    trans = bdata['trans']
    exps = bdata['expressions']
    

     
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)
    betas = torch.from_numpy(betas).float().to(comp_device)[::down_sample]
    poses = torch.from_numpy(poses).float().to(comp_device)[::down_sample]
    trans = torch.from_numpy(trans).float().to(comp_device)[::down_sample]
    exps = torch.from_numpy(exps).float().to(comp_device)[::down_sample]
    
    
    n, c = poses.shape[0], poses.shape[1]
    max_length = 128
    s, r = n//max_length, n%max_length
    #print(n, s, r)
    all_tensor = []
    with torch.no_grad():
        for i in range(s):
            with torch.no_grad():
                joints = smplx_bm(
                    betas=betas[i*max_length:(i+1)*max_length], 
                    transl=trans[i*max_length:(i+1)*max_length], 
                    expression=exps[i*max_length:(i+1)*max_length], 
                    jaw_pose=poses[i*max_length:(i+1)*max_length, 66:69], 
                    global_orient=poses[i*max_length:(i+1)*max_length,:3], 
                    body_pose=poses[i*max_length:(i+1)*max_length,3:21*3+3], 
                    left_hand_pose=poses[i*max_length:(i+1)*max_length,25*3:40*3], 
                    right_hand_pose=poses[i*max_length:(i+1)*max_length,40*3:55*3], 
                    return_verts=True,
                    return_joints=True,
                    leye_pose=poses[i*max_length:(i+1)*max_length, 69:72], 
                    reye_pose=poses[i*max_length:(i+1)*max_length, 72:75],
                )['joints'][:, target_joint, :]
            pose_seq.append(joints)
        if r != 0:
            with torch.no_grad():
                joints = smplx_bm(
                    betas=betas[s*max_length:s*max_length+r], 
                    transl=trans[s*max_length:s*max_length+r], 
                    expression=exps[s*max_length:s*max_length+r], 
                    jaw_pose=poses[s*max_length:s*max_length+r, 66:69], 
                    global_orient=poses[s*max_length:s*max_length+r,:3], 
                    body_pose=poses[s*max_length:s*max_length+r,3:21*3+3], 
                    left_hand_pose=poses[s*max_length:s*max_length+r,25*3:40*3], 
                    right_hand_pose=poses[s*max_length:s*max_length+r,40*3:55*3], 
                    return_verts=True,
                    return_joints=True,
                    leye_pose=poses[s*max_length:s*max_length+r, 69:72], 
                    reye_pose=poses[s*max_length:s*max_length+r, 72:75],
                )['joints'][:, target_joint, :]
            pose_seq.append(joints)

    pose_seq = torch.cat(pose_seq, dim=0)
    
    pose_seq_np = pose_seq.detach().cpu().numpy()
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)
    pose_seq_np_n[..., 0] *= -1
    np.save(save_path, pose_seq_np_n)
    return fps

In [ ]:

#all_count = sum([len(paths) for paths in group_path])
cur_count = 0


This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [ ]:
import time
pbar = tqdm(paths)
pbar.set_description('Processing:beat')
for path in pbar:
    fps = 0
    save_path = path.replace('./beat_data', './pose_data')
    save_path = save_path[:-3] + 'npy'
    fps = amass_to_pose(path, save_path)
    cur_count += len(paths)

In [ ]:
a = np.load('')

The above code will extract poses from **BEATX** dataset, and put them under directory **"./pose_data"**